In [1]:
import src.SecurityReader
from src.PCapLocationParams import PCapLocationParams
from src.MarketDate import *
import src.view.DataView as dview

import pandas as pd
import sqlite3
import numpy


In [3]:
print([1,2,3,4,5][-3:-1])

[3, 4]


In [2]:
ev_info = pd.read_feather(PCapLocationParams(channel=340, date='2021-05-12').feather_file_path(name='event_info'))
ev_info['lst_delta'] = ev_info['ls_time'] - ev_info['t_time']

In [4]:
# ev_info = pd.read_feather(PCapLocationParams(channel=340, date='2021-05-12').feather_file_path(name='event_info'))
len(ev_info)

30929213

In [13]:
class PolygonCache:
    def __init__(self):
        self.book_by_date = {}
        self.bad_dates = []

poly_market_dates = {pid: get_feather_dates(340, name='synthetic_book_4', month=[5,6]) for pid in range(1,12)}
feather_cache = {}
for pid, mds in poly_market_dates.items():
    poly_cache = PolygonCache()
    feather_cache[pid] = poly_cache
    for md in mds:
        fpath = PCapLocationParams(channel = 340, date=md).feather_file_path(f'synthetic_book_{pid}')
        try:
            fc = pd.read_feather(fpath)
            poly_cache.book_by_date[md] = fc
        except Exception as e:
            print(f'Error reading \'{fpath}\': {e}')
            poly_cache.bad_dates.append(md)

Error reading '/home/john/spartan_store/pcaps/340/2021-06-02_340_24h_feather/synthetic_book_4.feather': Not an Arrow file
Error reading '/home/john/spartan_store/pcaps/340/2021-06-07_340_24h_feather/synthetic_book_4.feather': Not an Arrow file
Error reading '/home/john/spartan_store/pcaps/340/2021-06-22_340_24h_feather/synthetic_book_4.feather': Not an Arrow file
Error reading '/home/john/spartan_store/pcaps/340/2021-06-02_340_24h_feather/synthetic_book_5.feather': Not an Arrow file
Error reading '/home/john/spartan_store/pcaps/340/2021-06-07_340_24h_feather/synthetic_book_5.feather': Not an Arrow file
Error reading '/home/john/spartan_store/pcaps/340/2021-06-22_340_24h_feather/synthetic_book_5.feather': File is too small to be a well-formed file
Error reading '/home/john/spartan_store/pcaps/340/2021-06-02_340_24h_feather/synthetic_book_6.feather': Not an Arrow file
Error reading '/home/john/spartan_store/pcaps/340/2021-06-07_340_24h_feather/synthetic_book_6.feather': Not an Arrow file

In [12]:
for pid, pcache in feather_cache.items():
    bad_dates = "\n\t".join(pcache.bad_dates)
    print(f'{pid:02}: {bad_dates}')

01: 2021-05-02
	2021-05-03
	2021-05-04
	2021-05-05
	2021-05-06
	2021-05-09
	2021-05-10
	2021-05-11
	2021-05-12
	2021-05-13
	2021-05-16
	2021-05-17
	2021-05-18
	2021-05-19
	2021-05-20
	2021-05-23
	2021-05-24
	2021-05-25
	2021-05-26
	2021-05-30
	2021-05-31
	2021-06-01
	2021-06-02
	2021-06-06
	2021-06-07
	2021-06-08
	2021-06-09
	2021-06-10
	2021-06-13
	2021-06-14
	2021-06-15
	2021-06-16
	2021-06-17
	2021-06-20
	2021-06-21
	2021-06-22
	2021-06-23
	2021-06-24
	2021-06-27
	2021-06-28
	2021-06-29
	2021-06-30
02: 2021-05-02
	2021-05-03
	2021-05-04
	2021-05-05
	2021-05-06
	2021-05-09
	2021-05-10
	2021-05-11
	2021-05-12
	2021-05-13
	2021-05-16
	2021-05-17
	2021-05-18
	2021-05-19
	2021-05-20
	2021-05-23
	2021-05-24
	2021-05-25
	2021-05-26
	2021-05-30
	2021-05-31
	2021-06-01
	2021-06-02
	2021-06-06
	2021-06-07
	2021-06-08
	2021-06-09
	2021-06-10
	2021-06-13
	2021-06-14
	2021-06-15
	2021-06-16
	2021-06-17
	2021-06-20
	2021-06-21
	2021-06-22
	2021-06-23
	2021-06-24
	2021-06-27
	2021-06-28
	2021-06-2

In [2]:
book = pd.read_feather('/home/john/spartan_store/pcaps/340/2021-06-13_340_24h_feather/book_zc.feather')
# book = pd.read_feather('/home/john/spartan_store/pcaps/340/2021-06-13_340_24h_feather/book_2021-07-06_01-17.feather')
secs = pd.read_feather('/home/john/spartan_store/pcaps/340/2021-06-13_340_24h_feather/security.feather')
book = pd.merge(book, secs.drop(columns=['implied_matching']), on='sid')

In [3]:
secs.loc[secs['asset'] == 'ZS']

,sid,symbol,expiration,sec_type,cme_sec_type,sec_group,asset,implied_matching,match_algo,display_factor,cash,tick_size,min_price,n_legs,segment_id
1,87218,ZSH2-ZSX2,2022-03-14 17:01:00,SPREAD,FUT,ZS,ZS,True,K,1.0,50.0,0.25,0.0,2,70
8,53832,ZSU1-ZSF2,2021-09-14 17:01:00,SPREAD,FUT,ZS,ZS,True,K,1.0,50.0,0.25,0.0,2,70
22,45007,ZSN1-ZSK3,2021-07-14 17:01:00,SPREAD,FUT,ZS,ZS,True,K,1.0,50.0,0.25,0.0,2,70
53,45669,ZSF3-ZSX4,2023-01-13 18:01:00,SPREAD,FUT,ZS,ZS,True,K,1.0,50.0,0.25,0.0,2,70
71,32834,ZSK3-ZSX3,2023-05-12 17:01:00,SPREAD,FUT,ZS,ZS,True,K,1.0,50.0,0.25,0.0,2,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3614,100974,ZSK2-ZSX3,2022-05-13 17:01:00,SPREAD,FUT,ZS,ZS,True,K,1.0,50.0,0.25,0.0,2,70
3634,43200,ZSK3-ZSU3,2023-05-12 17:01:00,SPREAD,FUT,ZS,ZS,True,K,1.0,50.0,0.25,0.0,2,70
3635,85342,ZSH2-ZSK2,2022-03-14 17:01:00,SPREAD,FUT,ZS,ZS,True,K,1.0,50.0,0.25,0.0,2,70
3636,65594,ZSX1-ZSX3,2021-11-12 18:01:00,SPREAD,FUT,ZS,ZS,True,K,1.0,50.0,0.25,0.0,2,70


In [58]:
bk = book.loc[book['symbol'] == 'ZCN1']\
    .drop(columns=['sec_type', 'cme_sec_type','sec_group', 'asset', 'match_algo', 'display_factor', 'cash', 'tick_size', 'min_price', 'n_legs', 'segment_id'])
# bk = book.query('status == \'READY_TO_TRADE\'').copy()

In [41]:
bbo_cols = ['bid_q_1', 'bid_p_1', 'ask_p_1', 'ask_q_1', 'ibid_p_1', 'iask_p_1']

In [92]:
bk['bid_change'] = bk['bid_p_1'] != bk['bid_p_1'].shift(-1)
bk['ask_change'] = bk['ask_p_1'] != bk['ask_p_1'].shift(-1)
bk['ibid_change'] = bk['ibid_p_1'] != bk['ibid_p_1'].shift(-1)
bk['iask_change'] = bk['iask_p_1'] != bk['iask_p_1'].shift(-1)
bk['bbo_change'] = bk['bid_change'] | bk['ask_change']
bk['ibbo_change'] = bk['ibid_change'] | bk['iask_change']
# bk
# bk.loc[(bk['status'] == 'READY_TO_TRADE') | bk['bbo_change']]
bk.loc[(bk['status'] == 'READY_TO_TRADE') & (bk['bbo_change'] == False)][['eid', 't_time'] + bbo_cols]
bk.

,eid,t_time,bid_q_1,bid_p_1,ask_p_1,ask_q_1,ibid_p_1,iask_p_1
2652507,10919,2021-06-14 00:00:00.000000000,157,675.00,676.0,3,675.75,676.00
2652508,10926,2021-06-14 00:00:00.003092857,157,675.00,676.0,3,675.75,676.00
2652509,10978,2021-06-14 00:00:00.010129709,157,675.00,676.0,3,675.75,676.00
2652510,10982,2021-06-14 00:00:00.012716403,157,675.00,676.0,3,675.50,676.00
2652513,11086,2021-06-14 00:00:00.165261247,157,675.00,676.0,3,675.00,676.00
...,...,...,...,...,...,...,...,...
3763333,23175613,2021-06-14 18:19:59.757932895,3,658.25,659.0,5,658.00,658.75
3763334,23175614,2021-06-14 18:19:59.757970129,3,658.25,659.0,5,658.00,658.75
3763335,23176067,2021-06-14 18:19:59.817241009,3,658.25,659.0,5,658.00,658.75
3763336,23176072,2021-06-14 18:19:59.827518161,3,658.25,659.0,5,658.00,658.75


In [4]:
pcap = PCapLocationParams(340,  '2021-06-09')
bbo = pd.read_feather('/home/john/spartan_store/pcaps/340/2021-06-09_340_24h_feather/synthetic_leg_book_2021-07-02_05-26.feather')
bbo.loc[bbo['sid'] == 24491]
# src.SecurityReader.read_pcap_securities(340, '2021-06-09')

,eid,t_time,sid,status,implied_matching,bid_p_1,bid_q_1,bid_no_1,ibid_p_1,ibid_q_1,ask_p_1,ask_q_1,ask_no_1,iask_p_1,iask_q_1
0,0,2021-06-09 21:39:59.568501189,24491,NOT_TRADING,False,38.75,4,1,0.00,0,39.00,20,5,0.00,0
6,8382,2021-06-09 23:53:58.308255133,24491,NOT_TRADING,False,38.75,5,2,0.00,0,39.00,18,4,0.00,0
7,9137,2021-06-09 23:55:00.434099873,24491,NOT_TRADING,False,38.75,14,4,0.00,0,39.00,26,5,0.00,0
8,9140,2021-06-09 23:55:00.434142675,24491,NOT_TRADING,False,38.75,33,6,0.00,0,39.00,35,6,0.00,0
17,11666,2021-06-10 00:00:00.000000000,24491,READY_TO_TRADE,True,38.75,33,6,35.75,2,39.00,31,6,40.25,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285226,25291787,2021-06-10 18:19:59.301273797,24491,READY_TO_TRADE,True,33.25,29,9,32.75,3,33.50,21,1,33.75,3
285239,25293322,2021-06-10 18:20:00.000000000,24491,NOT_TRADING,False,33.25,29,9,0.00,0,33.50,21,1,0.00,0
285242,25295902,2021-06-10 18:20:30.000011705,24491,NOT_TRADING,False,33.25,27,8,0.00,0,33.75,2,2,0.00,0
285244,25298041,2021-06-10 18:20:30.000319531,24491,NOT_TRADING,False,33.25,23,4,0.00,0,33.75,1,1,0.00,0


In [ ]:
secs

In [7]:
secs = pd.read_feather('~/spartan_store/pcaps/360/2020-09-07_360_24h_feather/security.feather')
books = pd.read_feather('~/spartan_store/pcaps/360/2020-09-07_360_24h_feather/book_2021-7-2459397_18-28.feather')
books = pd.merge(books, secs[['sid', 'symbol', 'sec_type']], on='sid')
bbo = books[['sid', 'bid_q_1', 'bid_p_1', 'ask_p_1', 'ask_q_1']]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7455 entries, 0 to 7454
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   sid               7455 non-null   int32         
 1   symbol            7455 non-null   object        
 2   expiration        7455 non-null   datetime64[ns]
 3   sec_type          7455 non-null   category      
 4   cme_sec_type      7455 non-null   category      
 5   sec_group         7455 non-null   category      
 6   asset             7455 non-null   category      
 7   implied_matching  7455 non-null   bool          
 8   match_algo        7455 non-null   category      
 9   display_factor    7455 non-null   float32       
 10  cash              7455 non-null   float32       
 11  tick_size         7455 non-null   float32       
 12  min_price         7455 non-null   float32       
 13  n_legs            7455 non-null   uint8         
 14  segment_id        7455 n

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26112472 entries, 0 to 26112471
Data columns (total 73 columns):
 #   Column            Dtype         
---  ------            -----         
 0   eid               uint32        
 1   t_time            datetime64[ns]
 2   sid               int32         
 3   status            category      
 4   implied_matching  bool          
 5   bid_p_1           float32       
 6   bid_q_1           uint32        
 7   bid_no_1          uint32        
 8   bid_p_2           float32       
 9   bid_q_2           uint32        
 10  bid_no_2          uint32        
 11  bid_p_3           float32       
 12  bid_q_3           uint32        
 13  bid_no_3          uint32        
 14  bid_p_4           float32       
 15  bid_q_4           uint32        
 16  bid_no_4          uint32        
 17  bid_p_5           float32       
 18  bid_q_5           uint32        
 19  bid_no_5          uint32        
 20  bid_p_6           float32       
 21  bid_q_